In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [18]:
#(x_train, y_train), (x_test, y_test)= keras.datasets.fashion_mnist.load_data()


train_df = pd.read_csv('')
#test_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')
#train_df.head()

In [19]:
train_data=np.array(train_df)# ,dtype='float32')
#test_data=np.array(test_df)# ,dtype='float32')

In [20]:
x_train= x_train[:,1:]/255
y_train= y_train[:,0]

x_test= x_test[:,1:]/255
y_test= y_test[:,0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
x_train.shape , y_train.shape

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
## Visualization
plt.figure(figsize=(16,16))
j = 1
for i in np.random.randint(0,1000,25):
  plt.subplot(5,5,j); j+=1
  plt.imshow((x_train[i].reshape((28,28))),cmap = 'Greys')
  plt.axis('off')
  plt.title('{} / {}'.format(class_names[y_train[i]],y_train[i]))

In [ ]:
## data split
x_train,x_validate,y_train,y_validate=train_test_split(
    x_train,y_train,test_size=0.2)

In [ ]:
im_rows = 28
im_cols = 28
batch_size = 512
im_shape = (im_rows,im_cols,1)

x_train= x_train.reshape(x_train.shape[0],*im_shape)
x_test= x_test.reshape(x_test.shape[0],*im_shape)
x_validate= x_validate.reshape(x_validate.shape[0],*im_shape)

In [ ]:
print('x train shape:',format(x_train.shape))
print('x test shape:',format(x_test.shape))
print('x validate shape:',format(x_validate.shape))

In [ ]:
## CNN model
cnn_model= Sequential([
    Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=im_shape),
    MaxPooling2D(pool_size=2),
    Dropout(0.2),
    
    Flatten(),
    Dense(32,activation='relu'),
    Dense(10,activation='softmax')
])

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [ ]:
history=cnn_model.fit(
    x_train, y_train, batch_size=batch_size,
    epochs=10, verbose=1,
    validation_data=(x_validate, y_validate),
)

In [ ]:
## Accuracy of trainnig data sets
score = cnn_model.evaluate(x_train, y_train, verbose=1)

print('train loss: {:.4f}'.format(score[0]))
print('train acc: {:.4f}'.format(score[1]))

In [ ]:
loss = history.history['loss']
plt.plot(loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train error'], loc='upper right')
plt.savefig('plot_error.png')
plt.show()

In [ ]:
## Accuracy for Validation data sets
score = cnn_model.evaluate(x_validate, y_validate, verbose=1)

print('Validation loss: {:.4f}'.format(score[0]))
print('Validation acc: {:.4f}'.format(score[1]))

In [ ]:
predictions=cnn_model.predict(x_test).round(2)
predictions

In [ ]:
print(class_names[np.argmax(predictions[9])])
plt.matshow(x_test[9])

In [ ]:
## Accuracy for test data sets
score = cnn_model.evaluate(x_test, y_test, verbose=0)

print('test loss: {:.4f}'.format(score[0]))
print('test acc: {:.4f}'.format(score[1]))

In [ ]:
plt.figure(figsize=(20,35))
j = 1
for i in np.random.randint(0,1000,50):
  plt.subplot(10,5,j); j+=1
  plt.imshow((x_test[i].reshape((28,28))),cmap = 'Greys')
  plt.axis('off')

  plt.title('Actual = {} / {} \nPredicted = {} / {}'.format(class_names[y_test[i]], y_test[i], 
     class_names[np.argmax(predictions[i])],np.argmax(predictions[i])))

In [ ]:
## confusion matrix

plt.figure(figsize=(16,9))
predictions_labels = [ np.argmax(label) for label in predictions ]
predictions_labels
cm = confusion_matrix(y_test, predictions_labels)

sns.heatmap(cm, annot=True, fmt='d',xticklabels=class_names, yticklabels=class_names)